In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


clients = pd.read_csv('../data/Clients.csv', delimiter=';', encoding='latin1')
commandes = pd.read_csv('../data/Commandes.csv', delimiter=';', encoding='latin1')
envois_chunks = pd.read_csv('../data/Envois.csv', delimiter=';', encoding='latin1', chunksize=500000)
envois = envois_chunks.get_chunk()
produits = pd.read_csv('../data/Produits.csv', delimiter=';', encoding='latin1')

c:\Users\lelia\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\lelia\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Create age groups
bins = [25, 30, 35, 40, 45, 50, 55, 60, 65, 100]
labels = ['25-30', '30-35', '35-40', '40-45', '45-50', '50-55', '55-60', '60-65', '65+']
clients['Age Group'] = pd.cut(clients['Age'], bins=bins, labels=labels, include_lowest=True)

In [3]:
# merge clients & commandes
merged_clients_comm = pd.merge(clients, commandes, on='id_client', how='inner')

# merge commandes & produits
merged_comm_prod = pd.merge(commandes, produits, on='id_produit', how='inner')

# merge all three tables
merged_clients_comm_prod = pd.merge(merged_clients_comm, produits, on='id_produit', how='inner')
merged_clients_comm_prod['Montant_Produit'] = merged_clients_comm_prod['Montant_Produit'].str.replace(',', '.').astype(float)
merged_clients_comm_prod['Montant_Remise'] = merged_clients_comm_prod['Montant_Remise'].str.replace(',', '.').astype(float)

# Create validation set

In [11]:
data_df = merged_clients_comm_prod.copy()

# Count of orders by Boutique and Web per client
web_counts = data_df[data_df['Canal_Commande'] == 'Web'].groupby('id_client').size().reset_index(name='Web_Count')
boutique_counts = data_df[data_df['Canal_Commande'] == 'Boutique'].groupby('id_client').size().reset_index(name='Boutique_Count')

result = pd.merge(web_counts, boutique_counts, on='id_client', how='outer').fillna(0).astype(int)

# nombre total de commandes dans les 12, 6 et 3 derniers mois
count_orders_year = data_df[data_df['dCommande'] > '2022-07-01'].groupby('id_client').size().reset_index(name='Count_Orders_year')
count_orders_6 = data_df[data_df['dCommande'] > '2023-01-01'].groupby('id_client').size().reset_index(name='Count_Orders_6months')
count_orders_3 = data_df[data_df['dCommande'] > '2023-04-01'].groupby('id_client').size().reset_index(name='Count_Orders_3months')

# Total amount spent per id_client
amount_spent = data_df.groupby('id_client')['Montant_Produit'].sum().reset_index(name='Total_Amount_Spent')

# Count of products bought in their launch year
data_df['Year'] = str(data_df['dCommande'])[0:3]
count_orders_launch_year = data_df[data_df['Year'] == data_df['Annee_lancement_produit']].groupby('id_client').size().reset_index(name='Count_Orders_Launch_Year')

result = pd.merge(result, count_orders_year, on='id_client', how='inner').astype(int)
result = pd.merge(result, count_orders_3, on='id_client', how='inner').astype(int)
result = pd.merge(result, count_orders_6, on='id_client', how='inner').astype(int)
result = pd.merge(result, amount_spent, on='id_client', how='inner').astype(float)
result = pd.merge(result, count_orders_launch_year, on='id_client', how='inner').astype(int)

# Count of orders placed after a limit date
limit_date = '2023-01-01'

orders_after = data_df[data_df['dCommande'] >= limit_date]
order_counts = orders_after.groupby('id_client').size().reset_index(name='Orders_After')

result = pd.merge(result, order_counts, on='id_client', how='inner')

# Final merge 
dataset_af = pd.merge(clients, result, on='id_client', how='inner')

dataset_af = dataset_af.drop(['dPremierEnvoi', 'dPremiereCommande'], axis=1, errors='ignore')

dataset_af = pd.get_dummies(dataset_af, columns=['Segmentation','Civilite', 'ZoneGeographique'])
dataset_af = dataset_af.dropna()

dataset_af.head()

,id_client,Host,Pays,Age,Anciennete_Compte,Member_Programme_de_fid,A_ete_Membre_Programme_de_fid,Age Group,Web_Count,Boutique_Count,Count_Orders_year,Count_Orders_3months,Count_Orders_6months,Total_Amount_Spent,Count_Orders_Launch_Year,Orders_After


# Create training set

In [5]:
# Filter the data based on the date of the commandes
train_date = '2023-04-01'
data_bf = merged_clients_comm_prod.copy()[merged_clients_comm_prod['dCommande'] <= train_date]

# Count of orders by Boutique and Web per client
web_counts = data_bf[data_bf['Canal_Commande'] == 'Web'].groupby('id_client').size().reset_index(name='Web_Count')
boutique_counts = data_bf[data_bf['Canal_Commande'] == 'Boutique'].groupby('id_client').size().reset_index(name='Boutique_Count')

result = pd.merge(web_counts, boutique_counts, on='id_client', how='outer').fillna(0).astype(int)

# nombre total de commandes dans les 12, 6 et 3 derniers mois
count_orders_year = data_bf[data_bf['dCommande'] > '2022-04-01'].groupby('id_client').size().reset_index(name='Count_Orders_year')
count_orders_6 = data_bf[data_bf['dCommande'] > '2022-10-01'].groupby('id_client').size().reset_index(name='Count_Orders_6months')
count_orders_3 = data_bf[data_bf['dCommande'] > '2023-01-01'].groupby('id_client').size().reset_index(name='Count_Orders_3months')

# Total amount spent per id_client
amount_spent = data_bf.groupby('id_client')['Montant_Produit'].sum().reset_index(name='Total_Amount_Spent')

# Count of products bought in their launch year
data_bf['Year'] = str(data_bf['dCommande'])[0:3]
count_orders_launch_year = data_bf[data_bf['Year'] == data_bf['Annee_lancement_produit']].groupby('id_client').size().reset_index(name='Count_Orders_Launch_Year')

result = pd.merge(result, count_orders_year, on='id_client', how='inner').astype(int)
result = pd.merge(result, count_orders_3, on='id_client', how='inner').astype(int)
result = pd.merge(result, count_orders_6, on='id_client', how='inner').astype(int)
result = pd.merge(result, amount_spent, on='id_client', how='inner').astype(float)
result = pd.merge(result, count_orders_launch_year, on='id_client', how='inner').astype(int)

# Count of orders placed after a limit date
limit_date = '2023-01-01'

orders_after = data_bf[data_bf['dCommande'] >= limit_date]
order_counts = orders_after.groupby('id_client').size().reset_index(name='Orders_After')

result = pd.merge(result, order_counts, on='id_client', how='inner')

# Final merge 
dataset_bf = pd.merge(clients, result, on='id_client', how='inner')

dataset_af = dataset_af.drop(['dPremierEnvoi', 'dPremiereCommande'], axis=1, errors='ignore')

dataset_bf = dataset_bf.dropna()

dataset_bf = pd.get_dummies(dataset_bf, columns=['Segmentation','Civilite', 'ZoneGeographique'])

dataset_bf['Segmentation_Actifs 12-36 mois multi-acheteurs'] = 0
dataset_bf['Segmentation_Nouveaux - 6 mois'] = 0
dataset_bf['Segmentation_Ractivs - 6 mois'] = 0

print(dataset_bf.columns)

Index(['id_client', 'Host', 'Pays', 'Age', 'Anciennete_Compte',
       'dPremierEnvoi', 'dPremiereCommande', 'Member_Programme_de_fid',
       'A_ete_Membre_Programme_de_fid', 'Age Group', 'Web_Count',
       'Boutique_Count', 'Count_Orders_year', 'Count_Orders_3months',
       'Count_Orders_6months', 'Total_Amount_Spent',
       'Count_Orders_Launch_Year', 'Orders_After',
       'Segmentation_Actifs 12-36 mois multi-acheteurs',
       'Segmentation_Nouveaux - 6 mois', 'Segmentation_Ractivs - 6 mois'],
      dtype='object')


In [6]:
# Step 1: Preprocess the data

# Drop unnecessary columns
cols_to_drop = ['id_client', 'Host', 'Age Group', 'Pays']
dataset_bf = dataset_bf.drop(cols_to_drop, axis=1, errors='ignore')

print(dataset_bf.columns)

# Normalize numerical variables
scaler = StandardScaler()
numerical_cols = ['Web_Count', 'Boutique_Count', 'Count_Orders_year', 'Count_Orders_3months', 'Count_Orders_6months', 'Total_Amount_Spent', 
                  'Count_Orders_Launch_Year']
dataset_bf[numerical_cols] = scaler.fit_transform(dataset_bf[numerical_cols])

# Split the data into training and validation sets
train_data, test_data = train_test_split(dataset_bf, test_size=0.2, random_state=42)

# Step 2: Build the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 3: Train the model
train_inputs = train_data.drop('Orders_After', axis=1)
train_labels = train_data['Orders_After']
test_inputs = test_data.drop('Orders_After', axis=1)
test_labels = test_data['Orders_After']

model.fit(train_inputs, train_labels)

# Step 4: Test the model

# Preprocess the validation data 
test_data = test_data.drop(cols_to_drop, axis=1, errors='ignore')
test_data[numerical_cols] = scaler.fit_transform(test_data[numerical_cols])

# Use the trained model to make predictions on the validation data
predictions = model.predict(test_inputs)

# Get the actual labels from the validation dataset
accuracy = accuracy_score(test_labels, predictions)
accuracy

Index(['Age', 'Anciennete_Compte', 'dPremierEnvoi', 'dPremiereCommande',
       'Member_Programme_de_fid', 'A_ete_Membre_Programme_de_fid', 'Web_Count',
       'Boutique_Count', 'Count_Orders_year', 'Count_Orders_3months',
       'Count_Orders_6months', 'Total_Amount_Spent',
       'Count_Orders_Launch_Year', 'Orders_After',
       'Segmentation_Actifs 12-36 mois multi-acheteurs',
       'Segmentation_Nouveaux - 6 mois', 'Segmentation_Ractivs - 6 mois'],
      dtype='object')


ValueError: Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by StandardScaler.

In [ ]:
from sklearn.svm import SVC

# Step 1: Preprocess the data (same as before)

# Step 2: Build the model
model = SVC(kernel='linear', random_state=42)

# Step 3: Train the model (same as before)
model.fit(train_inputs, train_labels)

# Step 4: Test the model on 'dataset' (same as before)
predictions = model.predict(test_inputs)

# Calculate the accuracy of the model (same as before)
accuracy = accuracy_score(test_labels, predictions)
accuracy


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Step 1: Preprocess the data (same as before)

# Step 2: Build the model
model = GradientBoostingClassifier(random_state=42)

# Step 3: Train the model (same as before)
model.fit(train_inputs, train_labels)

# Step 4: Test the model on 'dataset' (same as before)
predictions = model.predict(test_inputs)

# Calculate the accuracy of the model (same as before)
accuracy = accuracy_score(test_labels, predictions)
accuracy


In [ ]:
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Preprocess the data (same as before)
input_size = train_inputs.shape[1]  # Using shape[1] to get the number of columns

# Convert the input data to tensors
train_inputs_tensor = torch.tensor(train_inputs.values, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels.values, dtype=torch.float32).view(-1, 1)  # Reshape the labels
test_inputs_tensor = torch.tensor(test_inputs.values, dtype=torch.float32)

# Step 2: Build the model
model = nn.Sequential(
    nn.Linear(input_size, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 1)  # Removed Sigmoid for regression
)

# Step 3: Train the model (same as before)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in zip(train_inputs_tensor, train_labels_tensor):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {running_loss/len(train_inputs_tensor)}")

# Step 4: Test the model on 'val_inputs_tensor'
with torch.no_grad():
    predictions = model(test_inputs_tensor)

# Convert predictions and labels to numpy arrays for easier evaluation
predictions_np = predictions.numpy()

# Evaluate your regression model using appropriate metrics (e.g., Mean Absolute Error, Mean Squared Error)
mae = mean_absolute_error(test_labels, predictions_np)
mse = mean_squared_error(test_labels, predictions_np)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')

In [ ]:
# Step 1: Preprocess the data

# Drop unnecessary columns
cols_to_drop = ['id_client', 'Host', 'Civilite', 'Age Group', 'ZoneGeographique', 'Pays', 'dPremierEnvoi', 'dPremiereCommande']
dataset_af = dataset_af.drop(cols_to_drop, axis=1, errors='ignore')

# Normalize numerical variables
scaler = StandardScaler()
numerical_cols = ['Web_Count', 'Boutique_Count', 'Count_Orders_year', 'Count_Orders_3months', 'Count_Orders_6months', 'Total_Amount_Spent', 
                  'Count_Orders_Launch_Year']
dataset_af[numerical_cols] = scaler.fit_transform(dataset_af[numerical_cols])

# Split in inputs and labels sets
val_inputs = dataset_af.drop('Orders_After', axis=1)
val_labels = dataset_af['Orders_After']

input_size = val_inputs.shape[1]  # Using shape[1] to get the number of columns

# Convert the input data to a tensor
val_inputs_tensor = torch.tensor(val_inputs.values, dtype=torch.float32)

# Step 2: Use the trained DL model to make predictions
with torch.no_grad():
    predictions = model(val_inputs_tensor)

# Step 3: Convert the predictions to the desired format
predictions_np = predictions.numpy()

mae = mean_absolute_error(val_labels, predictions_np)
mse = mean_squared_error(val_labels, predictions_np)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
